In [6]:
import numpy as np
import torch

In [7]:
if torch.cuda.is_available():
  device = torch.device("cuda")
  print("CUDA is available")
else:
  device = torch.device("cpu")
  print("CUDA is not available")

CUDA is not available


In [8]:
from urllib.request import urlopen
from PIL import Image
import timm

img = Image.open(urlopen(
    'https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/beignets-task-guide.png'
))

model = timm.create_model('resnext101_32x8d.fb_swsl_ig1b_ft_in1k', pretrained=True)
model = model.eval()

# get model specific transforms (normalization, resize)
data_config = timm.data.resolve_model_data_config(model)
transforms = timm.data.create_transform(**data_config, is_training=False)

output = model(transforms(img).unsqueeze(0))  # unsqueeze single image into batch of 1

top5_probabilities, top5_class_indices = torch.topk(output.softmax(dim=1) * 100, k=5)


/home/bhg/anaconda3/envs/deeplearning/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
from urllib.request import urlopen
from PIL import Image
import timm

img = Image.open(urlopen(
    'https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/beignets-task-guide.png'
))

model = timm.create_model(
    'resnext101_32x8d.fb_swsl_ig1b_ft_in1k',
    pretrained=True,
    num_classes=0,  # remove classifier nn.Linear
)
model = model.eval()

# get model specific transforms (normalization, resize)
data_config = timm.data.resolve_model_data_config(model)
transforms = timm.data.create_transform(**data_config, is_training=False)

output = model(transforms(img).unsqueeze(0))  # output is (batch_size, num_features) shaped tensor

# or equivalently (without needing to set num_classes=0)

output = model.forward_features(transforms(img).unsqueeze(0))
# output is unpooled, a (1, 2048, 7, 7) shaped tensor

output = model.forward_head(output, pre_logits=True)
# output is a (1, num_features) shaped tensor



In [17]:
from urllib.request import urlopen
from PIL import Image
import timm
import torch
import os
import glob

# 定义图像路径
train_data_path = '../data/train.csv'
train_dir = '../data/train'
test_dir = '../data/test'

# 创建模型
model = timm.create_model('resnext101_32x8d.fb_swsl_ig1b_ft_in1k', pretrained=True)
model = model.eval()

# 获取模型特定的变换（包括归一化和调整大小）
data_config = timm.data.resolve_model_data_config(model)
transforms = timm.data.create_transform(**data_config, is_training=False)


In [18]:

# 读取训练数据并调整大小
for class_dir in os.listdir(train_dir):
    class_path = os.path.join(train_dir, class_dir)
    if os.path.isdir(class_path):
        for img_path in glob.glob(os.path.join(class_path, '*.png')):
            img = Image.open(img_path)
            img = img.resize((224, 224))
            img.save(img_path)



In [20]:
# 对测试数据进行分类
for img_path in glob.glob(os.path.join(test_dir, '*.png')):
    img = Image.open(img_path)
    img = img.resize((224, 224))
    img_tensor = transforms(img).unsqueeze(0)  # unsqueeze single image into batch of 1

    # 获取模型输出
    output = model(img_tensor)
    top1_prob, top1_class = torch.topk(output.softmax(dim=1), k=1)

    # 获取预测的类别索引
    predicted_class = top1_class.item()

    print('here')

    # 输出预测结果
    print(f"Image: {img_path}, Predicted Class: {predicted_class}")

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act1): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act1): ReLU(inplace=True)
      (conv2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
      (bn2): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (drop_block): Identity()
      (act2): ReLU(inplace=True)
      (aa): Identity()
      (conv3): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_

: 